In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt

tf.random.set_seed(0)

n_input = 1
n_hidden = 20
n_output = 1

In [2]:
tf.keras.backend.set_floatx('float64')
tf.keras.backend.floatx()

'float64'

In [3]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(n_hidden, input_shape=(n_input, ), activation='tanh'))
model.add(tf.keras.layers.Dense(n_hidden, activation='tanh'))
model.add(tf.keras.layers.Dense(n_hidden, activation='tanh'))
model.add(tf.keras.layers.Dense(n_output, activation='linear'))

optimizer = tf.optimizers.Adam(learning_rate=0.1)

In [4]:
# @tf.function
def residual(y, x):
  kappa = 0.5
  dy_dx = tf.gradients(y, x)[0]
  d2y_dx2 = tf.gradients(dy_dx, x)[0]
  
  LHS = kappa * d2y_dx2
  RHS = dy_dx - dy_dx**2 - y * dy_dx
  res = LHS - RHS
  return res

# @tf.function
def boundary(y, x):
  dy_dx = tf.gradients(y, x)[0]
  bdy1 = y[0] - 0
  bdy2 = dy_dx[0] - 0.1
  return bdy1**2 + bdy2**2

@tf.function
def step(x):    
    with tf.GradientTape() as tape:
        y = model(x)        
        # dy_dx = tf.gradients(y, x)[0]
        # d2y_dx2 = tf.gradients(dy_dx, x)[0]        
        # LHS = kappa * d2y_dx2
        # RHS = dy_dx - dy_dx**2 - y * dy_dx
        # res = LHS - RHS

        # dy_dx = tf.gradients(y, x)[0]
        # bdy1 = y[0] - 0
        # bdy2 = dy_dx[0] - 0.1
        # bdy = bdy1**2 + bdy2**2
        
        bdy = boundary(y, x)
        res = residual(y, x)
        loss = tf.reduce_mean(res**2 + bdy)
    
    weights = model.trainable_weights
    grads = tape.gradient(loss, weights)
    optimizer.apply_gradients(zip(grads, weights))

    return loss

In [5]:
kappa = tf.constant(0.5, dtype=tf.float64)
x = tf.linspace(0.0, 3.0, 1000)
# x = tf.cast(x, dtype=tf.float64)
X = tf.reshape(x, (-1, 1))

losses = []
for i in range(1000):
    loss = step(X).numpy()
    losses.append(loss)
    if i % 10 == 0:
      print("{:03d} : {:08.6f}".format(i, sum(losses[-10:])/10))


TypeError: ignored

In [ ]:
@tf.function
def solution(c):
  return tf.gradients(model(c), c)[0]

In [ ]:
fig, axs = plt.subplots(1,1, figsize=(18,5))

domain = x.numpy()
exact = tf.exp(domain).numpy()
predict = solution(X).numpy().flatten()

# axs[0].plot(domain, exact, label='exact')
axs.plot(domain, predict, label='predict')
axs.legend()

# axs[1].plot(domain, exact - predict, label='residual')
# axs[1].legend()
# fig.savefig('residual.jpg',bbox_inches='tight')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize=(10,5))
plt.plot(np.log(losses))
plt.savefig('f.jpg',bbox_inches='tight')